In [128]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import duckdb as dd

# Load the data

df = pd.read_csv('C:/Projects/GitHub/stldata.io/csb_data/merged_csb_data.csv',dtype=str)

In [ ]:
#UPDATE DATE TYPES
df['DATETIMECLOSED'] = pd.to_datetime(df['DATETIMECLOSED'])
df['DATEINVTDONE'] = pd.to_datetime(df['DATETIMECLOSED'])
df['DATETIMEINIT'] = pd.to_datetime(df['DATETIMEINIT'])
df['DATECANCELLED'] = pd.to_datetime(df['DATECANCELLED']) 
#Create Speed to Resolution Time Delta, convert to float and round to 2 decimal places
df['SpeedToResolution'] = df['DATETIMECLOSED'] - df['DATETIMEINIT']
df['SpeedToResolution'] = df['SpeedToResolution'].dt.total_seconds()/60/60/24
df['SpeedToResolution'] = df['SpeedToResolution'].round(2)

df['DATETIMECLOSED'] = pd.to_datetime(df['DATETIMECLOSED']).round('D')
df['DATEINVTDONE'] = pd.to_datetime(df['DATEINVTDONE']).round('D')
df['DATETIMEINIT'] = pd.to_datetime(df['DATETIMEINIT']).round('D')
df['DATECANCELLED'] = pd.to_datetime(df['DATECANCELLED']).round('D')

#Drop columns that are not needed
df.drop(['EXPLANATION'] , axis=1, inplace=True)
df.drop(['SRX'] , axis=1, inplace=True)
df.drop(['SRY'] , axis=1, inplace=True)
df.drop(['PROBADDRESS'] , axis=1, inplace=True)
df.drop(['CITY'] , axis=1, inplace=True)

#CONVERT df['WARD'] TO INT, Replacing any non-numeric values with nulls
df['WARD'] = pd.to_numeric(df['WARD'], errors='coerce')
df['WARD'] = df['WARD'].fillna(0)

#10% of the rows are duplicates, so we will remove them
df.drop_duplicates(inplace=True)
df.head()


,CALLERTYPE,DATECANCELLED,DATEINVTDONE,DATETIMECLOSED,DATETIMEINIT,DESCRIPTION,GRANDPARENT_ID,GRANDPARENT_NODE,GROUP,NEIGHBORHOOD,...,PROBADDTYPE,PROBLEMCODE,PROBLEMSID,PROBZIP,PUBLICRESOLUTION,REQUESTID,STATUS,SUBMITTO,WARD,SpeedToResolution
0,WEB,NaT,2022-01-04,2022-01-04,2022-01-01,Not enough cntnrs,17,Trash/Debris,Trash/Debris,NaN,...,B,Not enough cntnrs,703,63116,"Due to employee shortage, the routes will be c...",1538315,CLOSED,"REFUSE,",02,2.524148
1,WEB,NaT,2022-05-11,2022-05-11,2022-01-01,COVID-19 Social Distancing Violation,1874,Citizen Requests,Miscellaneous,NaN,...,B,COVID Distance Viol,1926,63118,no evidence of violation,1538316,CLOSED,"HEALTH DIR OFC,",07,129.576609
2,WEB,NaT,2022-01-04,2022-01-04,2022-01-01,Container blocking,17,Trash/Debris,Trash/Debris,NaN,...,B,Cntnr blocking,709,63104,container not blocking,1538317,CLOSED,"REFUSE,",08,3.231983
3,INDETERMINATE,NaT,2022-01-04,2022-01-04,2022-01-01,Signal Damaged,1874,Citizen Requests,Traffic Signal,78.0,...,B,Signal Damaged,314,NaN,work completed,1538318,CLOSED,"TRAFFIC DIVISION, 511",13,3.085490
4,INDETERMINATE,NaT,2022-02-03,2022-02-03,2022-01-01,Light Damaged,1874,Citizen Requests,Street Lights,86.0,...,B,Light Damaged,319,NaN,work completed,1538319,CLOSED,"TRAFFIC DIVISION, 511",12,33.202435


In [130]:



df.describe()

,SpeedToResolution
count,300635.000000
mean,33.848069
std,82.506625
min,-274.563391
25%,0.878701
50%,4.854909
75%,24.842034
max,1110.890532


In [131]:
df.to_csv('C:/Projects/GitHub/stldata.io/csb_data/merged_csb_df.csv',index=False)

In [132]:
df.columns

Index(['CALLERTYPE', 'DATECANCELLED', 'DATEINVTDONE', 'DATETIMECLOSED',
       'DATETIMEINIT', 'DESCRIPTION', 'GRANDPARENT_ID', 'GRANDPARENT_NODE',
       'GROUP', 'NEIGHBORHOOD', 'PARENT_ID', 'PARENT_NODE',
       'PLAIN_ENGLISH_NAME_FOR_PROBLEMCODE', 'PRJCOMPLETEDATE', 'PROBADDTYPE',
       'PROBLEMCODE', 'PROBLEMSID', 'PROBZIP', 'PUBLICRESOLUTION', 'REQUESTID',
       'STATUS', 'SUBMITTO', 'WARD', 'SpeedToResolution'],
      dtype='object')